<a href="https://colab.research.google.com/github/velickovski/colaboratoryProjects/blob/main/Bitcoin_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###Во оваа лабораторика вежба фокусот е на time series предвидувања, односно предвидување на Close (BTC) вредноста во даденото податочно множество за Bitcoin. Ваша задача е со помош на decision tree и boosting методи да ја предвидите бараната вредност земајќи ги во предвид релевантните колони од податочното множество, но дополнително и вредноста на Close (BTC) во изминатите четири дена, бидејќи вредноста на Bitcoin-от во даден момент зависи од нејзината вредност во изминатите денови.

Hint : внимавајте како ќе го поделите множеството на train и test бидејќи се работи податоци во временска серија

##Симнете го податочното множество

In [ ]:
!gdown 14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO

Downloading...
From: https://drive.google.com/uc?id=14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO
To: /content/crypto_data.csv
100% 90.0k/90.0k [00:00<00:00, 72.8MB/s]


##Направете претпроцесирање на податоците

In [ ]:
import pandas as pd
df=pd.read_csv('/content/crypto_data.csv')
df

,Date,Close (BNB),Close (BTC),Close (USDT),Close (ETH)
0,09-11-17,1.99077,7143.580078,1.00818,320.884003
1,10-11-17,1.79684,6618.140137,1.00601,299.252991
2,11-11-17,1.67047,6357.600098,1.00899,314.681000
3,12-11-17,1.51969,5950.069824,1.01247,307.907990
4,13-11-17,1.68662,6559.490234,1.00935,316.716003
...,...,...,...,...,...
1748,23-08-22,299.03000,21528.090000,1.00000,1662.770000
1749,24-08-22,296.45000,21395.020000,1.00000,1657.060000
1750,25-08-22,301.58000,21600.900000,1.00010,1696.460000
1751,26-08-22,279.60000,20260.020000,1.00000,1507.780000


In [ ]:
data=df
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')

In [ ]:
data.head()

,Close (BNB),Close (BTC),Close (USDT),Close (ETH)
Date,,,,
2017-09-11,1.99077,7143.580078,1.00818,320.884003
2017-10-11,1.79684,6618.140137,1.00601,299.252991
2017-11-11,1.67047,6357.600098,1.00899,314.681000
2017-12-11,1.51969,5950.069824,1.01247,307.907990
2017-11-13,1.68662,6559.490234,1.00935,316.716003


In [ ]:
data = data.sort_values(by='Date', ascending=False)

In [ ]:
data

,Close (BNB),Close (BTC),Close (USDT),Close (ETH)
Date,,,,
2022-12-08,327.510000,24402.82000,1.000300,1957.250000
2022-12-07,220.464554,19323.91406,0.999343,1038.191650
2022-12-06,255.950882,26762.64844,0.999080,1445.216553
2022-12-05,269.407257,29047.75195,0.997609,1961.701538
2022-12-04,413.873932,40127.18359,1.000435,3030.376465
...,...,...,...,...
2017-05-12,2.595300,11916.70020,1.001970,463.281006
2017-04-12,2.491680,11657.20020,1.003670,470.204010
2017-03-12,2.227250,11323.20020,1.000660,465.852997


In [ ]:

for i in range(1, 5):
  data[f'Close (BTC) from {i} days ago'] = data['Close (BTC)'].shift(i)
data=data.drop('Close (BTC)', axis=1)
data.dropna(subset=data.columns,axis=0,inplace=True)

data.head(10)

,Close (BNB),Close (USDT),Close (ETH),Close (BTC) from 1 days ago,Close (BTC) from 2 days ago,Close (BTC) from 3 days ago,Close (BTC) from 4 days ago
Date,,,,,,,
2022-12-04,413.873932,1.000435,3030.376465,29047.75195,26762.64844,19323.91406,24402.82000
2022-12-03,372.730377,1.000081,2574.754150,40127.18359,29047.75195,26762.64844,19323.91406
2022-12-02,403.389740,1.000601,2917.362793,38904.01172,40127.18359,29047.75195,26762.64844
2022-12-01,487.007416,1.000290,3372.258301,42244.46875,38904.01172,40127.18359,29047.75195
2022-11-08,326.370000,1.000400,1889.660000,43949.10156,42244.46875,38904.01172,40127.18359
2022-11-07,225.858505,0.999225,1097.236572,24334.78000,43949.10156,42244.46875,38904.01172
2022-11-06,269.487000,0.999025,1529.663452,19970.55664,24334.78000,43949.10156,42244.46875
2022-11-05,269.962769,0.995872,2072.108643,28360.81055,19970.55664,24334.78000,43949.10156
2022-11-04,394.086884,1.000348,2981.052246,28936.35547,28360.81055,19970.55664,24334.78000


##Поделето го на тренирачко и тестирачко множество во сооднос 80:20

In [ ]:

t = df.copy()
t.drop('Close (BTC)', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(t[t.columns[1:]],  df['Close (BTC)'], test_size=0.2,shuffle=False)


In [ ]:
X_train.shape

(1402, 3)

In [ ]:
Y_train.shape

(1402,)

##Искористете соодветен Boosting модел за проблемот и тестирајте го моделот на тестирачкото множество

In [ ]:

from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.metrics import r2_score


In [ ]:
r2_xgb=r2_score(Y_test, y_pred)

In [ ]:
mse_xgb=mean_squared_error(Y_test, y_pred)

In [ ]:
mae_xgb=mean_absolute_error(Y_test,y_pred)

##Искористете соодветен Decision Tree модел за проблемот и тестирајте го моделот на тестирачкото множество

In [ ]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor()
regressor.fit(X_train, Y_train)

y_pred = regressor.predict(X_test)

In [ ]:
r2_dt=r2_score(Y_test, y_pred)

In [ ]:

mse_dt=mean_squared_error(Y_test, y_pred)

In [ ]:
mae_dt=mean_absolute_error(Y_test,y_pred)

##Споредете ги резултатите добиени од двата модели со соодветни метрики

In [ ]:
print(' DecisionTreeRegressor Metrics: ')
print ('MAE: ', mae_dt)
print ('MSE: ', mse_dt)
print('R^2:', r2_dt)

 DecisionTreeRegressor Metrics: 
MAE:  1505.8587795669516
MSE:  16069303.208221981
R^2: 0.9391041400711186


In [ ]:
print('XGBRegressor Metrics:')
print ('MAE:', mae_xgb)
print ('MSE: ', mse_xgb)
print('R^2:', r2_xgb)

XGBRegressor Metrics:
MAE: 1393.6203856421387
MSE:  10241665.369748026
R^2: 0.9611884216936337


##Bonus Task: Направете визуелизации кои го илустрираат процесот на предвидување на двата горенавередени модели